## MNIST Training with MXNet and Gluon

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.



In [1]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## Download training and test data

In [2]:
WORK_DIRECTORY = '/home/ec2-user/SageMaker/gluonts_sagemaker/data'

## Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [4]:
inputs = sagemaker_session.upload_data(path=WORK_DIRECTORY, key_prefix='data/gluonts')
print(inputs)

s3://sagemaker-cn-northwest-1-343958593302/data/gluonts


## Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. The `train` function will check for the validation accuracy at the end of every epoch and checkpoints the best model so far, along with the optimizer state, in the folder `/opt/ml/checkpoints` if the folder path exists, else it will skip the checkpointing. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [ ]:
!cat 'main.py'

## Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on a single c4.xlarge instance. 

In [8]:
hyperparameters = {'algo-name': 'DeepAR', 'freq': '1H', 'prediction-length': 3*24, 'context-length': 7*24, 'epochs': 2}

m = MXNet("main.py",
          source_dir=".",
          role=role,
          train_instance_count=1,
          train_instance_type="local", # "ml.c5.2xlarge"
          framework_version="1.6.0",
          py_version="py3",
          hyperparameters=hyperparameters)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [ ]:
m.fit(inputs)

Creating l3sve9mriq-algo-1-1jynp ... 
Creating l3sve9mriq-algo-1-1jynp ... done
Attaching to l3sve9mriq-algo-1-1jynp
l3sve9mriq-algo-1-1jynp | 2021-04-01 07:51:34,074 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
l3sve9mriq-algo-1-1jynp | 2021-04-01 07:51:34,076 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
l3sve9mriq-algo-1-1jynp | 2021-04-01 07:51:34,086 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1-1jynp"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"algo-name":"DeepAR","context-length":168,"epochs":2,"freq":"1H","prediction-length":72}', 'SM_USER_ENTRY_POINT': 'main.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1-1jynp","hosts":["algo-1-1jynp"]}', 'SM_INPUT_DATA_CONFIG': '{"training":{"TrainingInputMode":"File"}}', 'SM_OUTPUT_DATA_DIR': '/opt/ml/output/data', 'SM_CHANNELS': '["training"]', 'SM_CURRENT_HOST': 'algo-

After training, we use the MXNet object to build and deploy an MXNetPredictor object. This creates a SageMaker endpoint that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays. 

In [ ]:
# predictor = m.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor. 

In [ ]:
# from sagemaker.predictor import json_serializer, json_deserializer

# data = [{"start": "2020-11-01 00:00:00", "target": [0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.042, 0.028, 0.028, 0.099, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.042, 0.014, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.042, 0.028, 0.014, 0.014, 0.028, 0.042, 0.028, 0.014, 0.028, 0.042, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.042, 0.028, 0.028, 0.028, 0.028, 0.028, 0.042, 0.028, 0.042, 0.014, 0.028, 0.028, 0.042, 0.028, 0.028, 0.042, 0.028, 0.028, 0.028, 0.028, 0.042, 0.028, 0.028, 0.128, 0.07, 0.014, 0.028, 0.042, 0.014, 0.042, 0.028, 0.042, 0.028, 0.014, 0.028, 0.042, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.042, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.042, 0.028, 0.014, 0.028, 0.028, 0.014, 0.042, 0.028, 0.028, 0.028, 0.014, 0.028, 0.042, 0.028, 0.028, 0.028, 0.028, 0.014, 0.014, 0.028, 0.028, 0.028, 0.014, 0.014, 0.042, 0.014, 0.014, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.042, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.014, 0.014, 0.028, 0.014, 0.028, 0.014, 0.042, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.014, 0.014, 0.028, 0.028, 0.014, 0.014, 0.028, 0.028, 0.014, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.028, 0.014, 0.014, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.042, 0.014, 0.028, 0.042, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.014, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.028, 0.028, 0.028, 0.042, 0.014, 0.028, 0.014, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.042, 0.028, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.0, 0.014, 0.014, 0.028, 0.28500000000000003, 0.684, 0.6980000000000001, 0.726, 0.67, 0.712, 0.684, 0.7120000000000001, 0.684, 0.684, 0.726, 0.37, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.028, 0.17000000000000004, 0.441, 0.7120000000000001, 0.7120000000000001, 0.11299999999999999, 0.042, 0.541, 0.726, 0.726, 0.726, 0.341, 0.028, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.014, 0.199, 0.712, 0.726, 0.74, 0.726, 0.6980000000000001, 0.712, 0.712, 0.6980000000000001, 0.056, 0.641, 0.085, 0.014, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.042, 0.028, 0.028, 0.427, 0.641, 0.726, 0.726, 0.726, 0.08499999999999999, 0.028, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.028, 0.042, 0.028, 0.028, 0.042, 0.07, 0.084, 0.042, 0.028, 0.014, 0.028, 0.014, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.014, 0.14200000000000002, 0.11299999999999999, 0.056, 0.028, 0.028, 0.028, 0.084, 0.6970000000000001, 0.712, 0.712, 0.712, 0.37000000000000005, 0.028, 0.014, 0.014, 0.014, 0.028, 0.014, 0.014, 0.014, 0.014, 0.014, 0.014, 0.028, 0.042, 0.5700000000000001, 0.6980000000000001, 0.6980000000000001, 0.726, 0.698, 0.712, 0.498, 0.056, 0.042, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.028, 0.042, 0.14200000000000002, 0.74, 0.698, 0.698, 0.655, 0.6980000000000001, 0.6980000000000001, 0.712, 0.726, 0.712, 0.712, 0.355, 0.014, 0.014, 0.028, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.014, 0.028, 0.028, 0.028, 0.0, 0.74, 0.754, 0.74, 0.74, 0.74, 0.74, 0.726, 0.74, 0.712, 0.341, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.028, 0.028, 0.07, 0.754, 0.74, 0.74, 0.74, 0.726, 0.698, 0.74, 0.726, 0.712, 0.74, 0.74, 0.384, 0.028, 0.014, 0.028, 0.014, 0.028, 0.014, 0.014, 0.028, 0.014, 0.028, 0.028, 0.042, 0.127, 0.712, 0.754, 0.74, 0.74, 0.754, 0.754, 0.74, 0.768, 0.726, 0.726, 0.384, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.028, 0.028, 0.014, 0.042, 0.056, 0.056, 0.6260000000000001, 0.782, 0.768, 0.754, 0.754, 0.768, 0.754, 0.782, 0.768, 0.74, 0.768, 0.342, 0.042, 0.028, 0.042, 0.042, 0.028, 0.028, 0.042, 0.028, 0.014, 0.042, 0.042, 0.07, 0.5549999999999999, 0.768, 0.796, 0.768, 0.74, 0.768, 0.754, 0.782, 0.782, 0.754, 0.782, 0.37, 0.042, 0.042, 0.042, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.042, 0.028, 0.08499999999999999, 0.74, 0.754, 0.768, 0.768, 0.74, 0.754, 0.754, 0.768, 0.754, 0.74, 0.754, 0.327, 0.042, 0.028, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.042, 0.042, 0.07, 0.5549999999999999, 0.74, 0.754, 0.754, 0.726, 0.74, 0.754, 0.768, 0.754, 0.74, 0.782, 0.355, 0.042, 0.056, 0.028, 0.028, 0.028, 0.028, 0.042, 0.014, 0.028, 0.042, 0.042, 0.07, 0.441, 0.768, 0.74, 0.754, 0.7970000000000002, 0.782, 0.782, 0.768, 0.7550000000000001, 0.768, 0.768, 0.341, 0.042, 0.042, 0.028, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.028, 0.042, 0.056, 0.7250000000000001, 0.782, 0.768, 0.782, 0.7830000000000001, 0.7970000000000002, 0.796, 0.782, 0.782, 0.768, 0.754, 0.35500000000000004, 0.042, 0.028, 0.028, 0.028, 0.028, 0.014, 0.028, 0.014, 0.028, 0.028, 0.042, 0.056, 0.512, 0.768, 0.74, 0.754, 0.782, 0.768, 0.782, 0.7970000000000002, 0.726, 0.768, 0.74, 0.341, 0.042, 0.028, 0.014], "dynamic_feat": [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2.51378427767371, 2.40217405829641, 1.53665172981533, 0.399105142046723, 1.66592409984624, 2.78393473771948, 3.05374461112958, 2.80069623323001, 2.47797438579026, 2.42352509216838, 2.74894621994896, 3.37213487477697, 4.13682858043411, 4.85960529382429, 5.27178753686114, 5.1411260341076, 4.54549266399105, 3.59878440405658, 2.41784004406435, 1.41783805254468, 1.0741058717463, 1.17760864112006, 1.58770778718662, 2.13015754043785, 2.44364196954378, 2.59992001011746, 2.81596243112816, 3.20785765116959, 3.43956771013148, 3.05915393128348, 1.87751084729607, 0.880317831036692, 1.00738120518806, 1.46220311802377, 2.92558398257528, 4.47266436731234, 5.21241884261448, 5.17131863245747, 4.60812825371159, 3.77942915119537, 2.91032945174724, 2.19592443631852, 1.84179333585051, 1.89377372650089, 2.02454357703619, 1.87933586528648, 1.56292568206508, 1.29080473681409, 1.089062981606, 0.882647768958726, 0.679486490822588, 0.605115348840936, 0.670706520310408, 0.628989680748034, 0.459381232257684, 0.625614683980285, 0.773173804160843, 0.47819871578772, 0.418511233352289, 1.43141879217731, 2.64403939635358, 3.74682330602462, 4.40897653913156, 4.40210155180712, 3.92786067781935, 3.29687164807138, 2.76328397892733, 2.37956349157131, 2.13255251376079, 2.01270442182565, 2.00785301178086, 2.07088635387464, 2.16806328932646, 2.32718613866803, 2.50184412839051, 2.60488082471408, 2.62563399755016, 2.54659621148671, 2.34055252303903, 2.10236423212332, 1.94768987902084, 2.0045178245078, 2.33856547338039, 2.84803809091808, 3.35785703130112, 3.77015843044928, 4.02794550698154, 4.06580979128916, 3.76351203570391, 2.97321281321022, 1.67609003444692, 0.504386097016217, 0.729010427360214, 0.56146957172827, 0.169343778773154, 0.761097350418923, 0.894066002219053, 0.749716044644462, 0.719003296487797, 0.842108130903139, 0.964979377931499, 1.02713549904654, 1.02985146629947, 0.989226260118357, 0.862967604626543, 0.560402681794079, 0.147903674866043, 0.573317396520138, 1.25356732935971, 1.91556535433527, 2.44183105292038, 2.73733759736457, 2.79583318913007, 2.63864728075597, 2.32180550768135, 1.99240598598488, 1.77999603352632, 1.67278359085376, 1.40949077496668, 0.724708198867184, 0.630183471445485, 2.02038141682418, 3.03603092599638, 3.2917296616958, 3.10321893254781, 2.88329128042556, 2.88667555797776, 3.12675988152101, 3.57035805493941, 4.12411044081565, 4.72310645917218, 5.30860683872965, 5.79606365349412, 6.00834550561874, 5.74582114782957, 4.89209611683667, 3.62991685899543, 2.2261755508768, 1.45380293280163, 2.15637509757792, 2.7382495991555, 2.33546268095068, 1.35527875467438, 0.645052055344576, 0.845897286403014, 1.23444824932639, 1.68787165177623, 2.19323970790452, 2.63825440265995, 2.84383499483046, 2.69024896058925, 2.42414338430708, 2.39225818227032, 2.83525010746548, 3.56235859342913, 4.28268041837176, 4.76770360313735, 4.95491395428855, 4.77604186128904, 4.200756381817, 3.49426477053265, 3.00352151997187, 2.93374896463415, 2.99922181112249, 2.81428952063392, 2.18023877102336, 1.57953306364701, 1.53061937972578, 1.67906501735197, 1.75129389247825, 1.85054785834991, 2.02978006077548, 2.17962770666561, 2.17653667414216, 1.94781738966348, 1.5975935815708, 1.27188605257681, 1.05432230233705, 0.784240657499326, 0.237433884528723, 0.747671518978559, 1.84707862350827, 2.72952482619589, 3.22438800570714, 3.3944764398852, 3.05374761821064, 1.97778914312696, 0.655944321000872, 0.595647620621713, 1.13025419711592, 1.77752716469724, 2.19051390987467, 1.86685420098621, 1.12207889257276, 0.533181429290065, 0.453128266423678, 0.769002229324822, 1.19463624954695, 1.43685234117182, 1.44156325295823, 1.21580560409121, 0.785466433174051, 0.238254551911016, 0.536990965524318, 1.43787310164406, 2.26200545284751, 2.58044019090891, 2.22172274758951, 1.73565752010741, 1.39837848141668, 0.970083569809909, 1.43010909727564, 2.13205145334309, 2.17191687380379, 1.83212062071105, 1.67421507792986, 1.86695187548271, 2.13503461129127, 2.26954715858807, 2.13119259284065, 1.83045813892837, 1.5306882777537, 1.34262170278577, 1.24530387488351, 1.18347247498964, 1.10346653317173, 0.954316417302015, 0.657453769964397, 0.386261181229277, 0.870600415731574, 1.38291283530719, 1.57523923091026, 1.53110073666506, 1.4763097931563, 1.63608506763326, 1.92049461223961, 2.06425878756836, 1.89278157344388, 1.5762895816297, 1.37379806253887, 1.47016966180037, 1.7504329121379, 2.01428441265494, 2.10023448800117, 2.03901338781892, 1.91234099302327, 1.79615594386254, 1.69220641905678, 1.558455846238, 1.460623434795, 1.69476427863558, 2.37172198638581, 3.28723631916868, 4.12639697145792, 4.60359631967566, 4.54247315955904, 4.08293300035439, 3.45505340367455, 2.88775116445566, 2.54181780577679, 2.40104965874517, 2.31983039156322, 2.24864743003739, 2.20700107744784, 2.17054833045605, 2.1672546029456, 2.26506468327776, 2.48257643793877, 2.61521063953123, 2.41030458724352, 1.76279778689024, 1.08205733976062, 0.747882277261955, 0.422242164535902, 0.197246796038421, 0.136048387546713, 0.606447642165822, 1.72203740448379, 2.74667147024549, 3.2916894696504, 3.39069237286829, 3.20346579228905, 2.93363670332596, 2.74761378097048, 2.61660124009299, 2.41577725258687, 2.21346280199581, 2.14208053926404, 2.17527457434109, 2.22584431581877, 2.25241542555575, 2.25040169818777, 2.22631061019418, 2.17454465336921, 2.0753174811979, 1.92606391977812, 1.73502446941545, 1.50218218904437, 1.1911357720325, 0.756895243413029, 0.221082902286617, 0.421822879435947, 0.858971936084652, 1.35191520844353, 2.02125638320119, 2.6238454688543, 2.87989064814457, 2.80333201775279, 2.50702532279307, 2.12708331001082, 1.79825580180909, 1.56993259824088, 1.37368424043429, 1.17705197277518, 1.03934708631501, 1.0263637729937, 1.08815774546232, 1.12715192149609, 1.06507337899185, 0.878910115823644, 0.55985778935319, 0.47475012158553, 1.05110379383566, 1.67718074822601, 2.11388766419929, 2.3453020675491, 2.38626091480834, 2.2897963698725, 2.11602374039246, 1.82464200370467, 1.34362244895222, 0.999187803238103, 1.15247043876987, 1.48133340267507, 1.69652449332477, 1.70524384004921, 1.46174424508879, 1.15085531424428, 0.966502235531025, 0.958056763075315, 1.07364887710316, 1.22021894818715, 1.25952955679412, 1.18047760891413, 1.02520022781721, 0.884852439711439, 0.921133684702994, 1.21264342625806, 1.65750072224919, 2.12566907156613, 2.51242913704644, 2.75380918880804, 2.86413112814333, 2.88003214526798, 2.83589815321112, 2.7569544465002, 2.65357430423421, 2.51800266971235, 2.32499016218993, 2.03589326343597, 1.64506100738873, 1.30867402630586, 1.22234720902426, 1.51057285988903, 1.94355559949244, 2.18153883531792, 1.99187809822214, 1.50945981303696, 0.942548931261924, 1.29329730967916, 2.4408575363658, 3.36794170151811, 3.93026788353833, 4.50703949544509, 4.723965323884, 4.1720427407112, 3.69784747870616, 4.00228451272103, 4.37797742402453, 4.41023745392195, 4.19745273827618, 3.87559827460537, 3.50077963209536, 3.11795404234599, 2.76824719739961, 2.45233015858034, 2.14334542705074, 1.81820055379294, 1.50956098053457, 1.297146825654, 1.25729571868356, 1.36852582331184, 1.51790301482058, 1.61187092282292, 1.66302519358725, 1.72475815477032, 1.84518515807007, 2.01994039486261, 2.2282571108333, 2.445856646917, 2.6460846903826, 2.80126861502347, 2.88372042538182, 2.86657129431368, 2.72499949679753, 2.467475491792, 2.21195921134998, 2.07708920793778, 2.14246335933223, 2.34833936481786, 2.5496219908352, 2.60361668355225, 2.52026368900466, 2.346932887843, 2.11833512603166, 1.8928584164268, 1.73966707163938, 1.66863509859637, 1.44722841067801, 0.793805691079637, 0.531928779684809, 1.75921354238095, 2.41744833861419, 2.40654835032481, 2.7974675785682, 3.49505498074427, 3.56381078381799, 3.07317776793988, 2.49703052807571, 2.22670749253554, 2.24512107175654, 2.44658991358982, 2.74066587639599, 3.09212125123687, 3.47065098714763, 3.83818368904097, 4.15004781007291, 4.36294820520781, 4.43556290859491, 4.32785273898254, 4.00125329320745, 3.4490318139747, 2.78314085360308, 2.14368473876848, 1.64019964052434, 1.29149535194226, 1.10103287910662, 1.07200716576537, 1.20002141090353, 1.48236376270912, 1.91136626374114, 2.41672210492609, 2.91422513926447, 3.32097294859587, 3.5542508076349, 3.53127560227795, 3.20785546675823, 2.69444220839531, 2.14020342666307, 1.66886600512259, 1.30147256935094, 1.03715750808376, 0.878961105305086, 0.825852856272725, 0.861531257070301, 0.958527942467423, 1.09247011179522, 1.24709317789925, 1.41161213369928, 1.57759380776215, 1.73721860453171, 1.88255881420491, 2.00536117591504, 2.09704029325818, 2.14874899496192, 2.15147640282278, 2.09617062815399, 1.97392502390389, 1.77635875899713, 1.49665906782123, 1.15406722148643, 0.859326264314549, 0.725726354381527, 0.756878975879162, 0.795425637530133, 0.676225273977104, 0.312720164079902, 0.506232186243885, 1.23255497735855, 1.97811376366357, 2.65369110662137, 3.20126760979248, 3.57212086127756, 3.74222969012866, 3.69400390454008, 3.38682707535867, 2.68037073467151, 1.4206558929142, 0.769454968146613, 2.64784481676317, 4.01177693641352, 4.29100314142052, 3.8703510395494, 3.30758017792971, 2.96172788630588, 2.79215327910218, 2.6740267400228, 2.47592532139002, 2.27244976512513, 2.21002935557874, 2.392482596336, 2.74919383627167, 3.17314807689231, 3.5909441497204, 3.91732900219622, 3.98578767375504, 3.67405245154507, 3.17168457707922, 2.71660995020316, 2.55225704394525, 2.71177739003772, 2.82052926715837, 2.45736445998136, 1.80722383140608, 1.31417068528039, 1.38018714735528, 1.75702183468276, 2.07919876854409, 2.15411581830305, 2.07695436331026, 2.01093709641584, 2.07126672668374, 2.18956788873791, 2.25290223053853, 2.19630708910975, 2.13426228156752, 2.21986796465331, 2.5055851938224, 2.68302149404487, 2.35769152669465, 1.29246929056713, 0.220861352936811, 1.47374062573511, 2.2899327044567, 2.69970628744419, 2.77459103528007, 2.51964241922714, 2.08836984123142, 1.70408177294093, 1.53749513308254, 1.53728251702384, 1.58630848708931, 1.58300332134113, 1.53735183426182, 1.49207706916702, 1.47989083183467, 1.45843156766308, 1.36218228270553, 1.17971491442702, 1.00105844490852, 0.855097641059655, 1.29392174591265, 2.22210699494036, 2.84205279559413, 2.69642745616362, 2.09230816139284, 1.50824458614426, 1.28614936230226, 1.36265636526977, 1.57316884784634, 1.76524770476933, 1.94762944261418, 2.10298364992233, 2.19525200363947, 2.16785821546407, 1.90911126983716, 1.33925762326532, 0.732612627228368, 0.486708912486764, 0.518028941761072, 0.678355209048748, 0.789308566850081, 0.630474706471589, 0.717502059167695, 1.51265033160355, 2.57912742778507, 3.46685712866401, 3.75386553112347, 3.21609647894252, 2.33382531975001, 1.68282360312434, 1.46582570061303, 1.55746700122553, 1.79545082927267, 1.92587046576608, 2.039369904865, 2.13866980445883, 2.1407585240326, 2.03647403343736, 1.75581575904124, 1.17615494850811, 0.545395396753924, 0.249542791014441, 0.497291182093364, 0.968636393144097, 1.1368498065044, 0.78751726195147, 0.858016307878826, 1.25687646254019, 1.11764479665457, 0.726799179111759, 0.661807907658408, 1.32264064557982, 2.30208647493934, 3.01815893337828, 3.08707707448298, 2.72694595713869, 2.13093106689501, 1.44068537794552, 1.24938003364192, 1.56312439853398, 1.69567106057454, 1.88647557781026, 2.12904977348547, 2.10713410007034, 2.00533370640339, 2.05358815953368, 2.15825682425155, 2.15106906661576, 1.99209489881935, 1.70686639575061, 1.51631278572949, 1.72651113958268, 2.46017104044114, 3.27436817107727, 3.63291069350477, 3.19667671440106, 2.34286856961858, 1.6228644088147, 1.51743246417894, 1.894074133588, 2.27925374646884, 2.32653795411587, 2.1305710471186, 1.89468357013004, 1.7651439201899, 1.72623711555424, 1.72829867643426, 1.73696736514964, 1.75615559666165, 1.80107816643405, 1.87320499096693, 1.9134806594677, 1.85067805366001, 1.64444942096637, 1.33724463751565, 0.980446648487106, 0.631426364608865, 0.670900281220576, 1.38699335096173, 2.43294410005015, 3.43205246269193, 4.02706421296602, 4.00073731448024, 3.57691622175918, 3.06629961336382, 2.66473223309906, 2.38546680340226, 2.22486318897081, 2.18110797403465, 2.19906490514967, 2.22134482898401, 2.20487535158869, 2.1164930067294, 1.90794943012526, 1.53876596309225, 1.06709674027039, 0.59190538608542, 0.282923122889116, 0.426636037629986, 0.717988582850996, 0.986846435199613, 1.21613412787113, 1.40576820051491, 1.55872414037938, 1.67872758856589, 1.76962245366936, 1.8351159727706, 1.87863394169011, 1.90321661014089, 1.91143780121467, 1.90534807947012, 1.88645136719865, 1.85572989924068, 1.8137377124274, 1.76079096891598, 1.69847300106216, 1.63384166206221, 1.57638876712437, 1.5363148607241, 1.52320013620931, 1.54407014717035, 1.60163697413734, 1.69377995787297, 1.81455773604789, 1.9559652442252, 2.109458914133, 2.26684184939942, 2.42060423114808, 2.56395702372257, 2.69074108381708, 2.79530861795907, 2.87241760364189, 2.91715238343438, 2.92487362432412, 2.89120006729735, 2.81203001184592, 2.68362453232553, 2.50280810080631, 2.26743154366184], [3.58511119689886, 2.78429193993083, 2.34279785156252, 2.46840384786341, 2.98572227322083, 3.62357482910158, 4.19802423901306, 4.87409731462518, 5.90406188964846, 7.44173647364521, 9.24714227958497, 10.981851196289, 12.3405341072192, 13.150257874399, 13.2711883544922, 12.6548928761958, 11.6185446563403, 10.570718383789, 9.82956509202646, 9.35154119302033, 9.00267944335939, 8.63571380756469, 8.0501830818866, 7.03232727050783, 5.48150627276696, 3.74955956862608, 2.30144653320314, 1.47768596935079, 1.12103439086034, 0.949807739257835, 0.805036498050493, 1.0186093186705, 2.04512939453127, 4.13749529991293, 6.74178713256252, 9.10238037109377, 10.6120047813198, 11.2568072778518, 11.1712890625, 10.4997744067304, 9.42587986063102, 8.14304504394533, 6.83543260333263, 5.65009729311524, 4.72481689453127, 4.14821708568282, 3.81231513212803, 3.55997619628908, 3.26788932990555, 2.94803914198674, 2.64623413085939, 2.38677613632559, 2.10794036408887, 1.72649536132814, 1.2530983014415, 1.07396086270388, 1.66918334960939, 3.32988610495712, 5.59126962476586, 7.79955444335939, 9.41907808584096, 10.3866460404318, 10.7571807861328, 10.5906791513247, 9.96743536190819, 8.97281799316408, 7.71931027124463, 6.42785402578821, 5.34650573730471, 4.63532479950417, 4.10238225889739, 3.46775207519533, 2.54385773903596, 1.51252086476528, 0.647912597656272, 0.166914532438024, 0.0572500616640052, 0.249353027343772, 0.712918764820285, 1.57468858277009, 3.00066528320314, 5.07476887905643, 7.55258822697508, 10.1076293945312, 12.4114468358522, 14.1277885512857, 14.9184509277343, 14.6036592497206, 13.6373543922455, 12.6319061279297, 12.0312172132614, 11.6053223401994, 10.9557891845703, 9.82307158346792, 8.50316801905596, 7.43096313476564, 6.9250588488812, 6.83892617909907, 6.90975341796877, 6.90610654097267, 6.72206225532475, 6.28307495117189, 5.62857330066863, 5.25388310400017, 5.76830444335939, 7.53875711218335, 9.96263974888588, 12.194970703125, 13.5721694637057, 14.1562600760206, 14.1906677246093, 13.8831918188373, 13.2991286673731, 12.4681488037109, 11.4572836295068, 10.483008019063, 9.79915771484377, 9.56815744586845, 9.58678788737876, 9.56041870117189, 9.25513591246801, 8.67989100017922, 7.90435180664064, 7.00765940049806, 6.10684775563999, 5.32842407226564, 4.80246930599704, 4.67335943414749, 5.08904418945314, 6.12819147971674, 7.59234191300742, 9.21375427246096, 10.7234493092796, 11.847495646932, 12.3107238769531, 11.91840265415, 10.7975528864174, 9.15563354492189, 7.25369871850677, 5.56718296672266, 4.62511596679689, 4.77383553027202, 5.62891200595129, 6.62322387695314, 7.27920317895995, 7.4774961579098, 7.18830261230471, 6.41763987599477, 5.31479542422242, 4.06487426757814, 2.90848893110187, 2.3082819976312, 2.78240356445314, 4.62971283510856, 7.27190543815365, 9.91138610839846, 11.8815248588531, 13.0395528153286, 13.3736663818359, 12.9023729960442, 11.7654242302554, 10.1328826904297, 8.20980927911261, 6.34125808519185, 4.90728149414064, 4.16080721877715, 3.84626428129931, 3.58095703125002, 3.05468584480234, 2.24723520465008, 1.21088562011721, 0.0331025550445562, -1.05790870865964, -1.79875793457028, -1.94951095484588, -1.36405787597926, 0.0802551269531477, 2.39932892348264, 5.18205256157134, 7.91056213378908, 10.1185217131, 11.5457072940458, 11.9834228515625, 11.3667424182, 10.2058202569646, 9.15458068847658, 8.65482900345909, 8.29989437304664, 7.47098693847658, 5.79667065325019, 3.89492471992448, 2.63108215332033, 2.59079203118201, 3.24096768294711, 3.76883850097658, 3.55450267529154, 2.74953358655395, 1.69837341308596, 0.777518610967899, 0.491682747312882, 1.37763366699221, 3.72638538264544, 6.84593657798515, 9.79853210449221, 11.8391844602299, 12.993976729592, 13.4817596435547, 13.4674529935209, 12.900252812602, 11.6754241943359, 9.82241982997299, 7.90744280161163, 6.63088378906252, 6.43785025765924, 6.75231681482742, 6.74297485351564, 5.84272328363205, 4.54129108292285, 3.59261474609377, 3.51649667942725, 3.89620293551255, 4.08086547851564, 3.6704594910274, 3.26833302933839, 3.72867736816408, 5.61960042409924, 8.3648766812558, 11.102197265625, 13.1309325589588, 14.3971699660522, 15.0086761474609, 15.0496264481745, 14.5098309509182, 13.3555084228515, 11.6480437446973, 9.82948625143353, 8.43705139160158, 7.83856514566339, 7.72429562176204, 7.61512145996096, 7.15127888843333, 6.45043448779156, 5.74961242675783, 5.22127935648448, 4.77967185784441, 4.27446899414064, 3.67689284759354, 3.44433757609277, 4.15574035644533, 6.1644913348472, 8.92179253505025, 11.6532989501953, 13.7184776103629, 15.0120436933923, 15.5625244140625, 15.4144695887184, 14.6765194399697, 13.4733367919922, 11.9593878046564, 10.4083519806119, 9.12371215820314, 8.32477509561455, 7.89414323039208, 7.63024291992189, 7.34885185556485, 6.93515306458051, 6.29168090820314, 5.39568210162848, 4.52325277589824, 4.02520141601564, 4.1849681772298, 5.01651989577339, 6.46645507812502, 8.43990510065759, 10.6761328193213, 12.8729339599609, 14.7463504121104, 16.0854087200618, 16.6973815917968, 16.453882749237, 15.4838899700637, 13.9807220458984, 12.1428820458025, 10.189610148598, 8.34533081054689, 6.79527975946532, 5.56793780938666, 4.65259704589846, 4.02260355319965, 3.58751940993469, 3.24096069335939, 2.90096283835538, 2.58323871030755, 2.32792053222658, 2.22997695553893, 2.60372234533406, 3.81830749511721, 6.06763969738108, 8.8446522405684, 11.4670349121093, 13.3697205799875, 14.456614434399, 14.7488647460937, 14.3005759870454, 13.2976774341255, 11.9590545654297, 10.5067504962447, 9.17543889062219, 8.20295104980471, 7.74626846352251, 7.63897337545608, 7.63379821777346, 7.53768060933691, 7.37437891578566, 7.22185668945314, 7.13424564878995, 7.07035017671532, 6.96514282226564, 6.80612168500715, 6.79088706662544, 7.16956481933596, 8.1050806582374, 9.41155974996155, 10.8159271240234, 12.0579926214241, 12.9291053291086, 13.2334991455078, 12.8653878887266, 12.0789050555658, 11.2181640625, 10.5271630900175, 9.84943937466242, 8.92841491699221, 7.63043114153872, 6.31350716873014, 5.45858154296877, 5.39437349743732, 5.84072502044027, 6.36525878906252, 6.61235413846037, 6.53341703607573, 6.15661010742189, 5.59550833829739, 5.30533615564174, 5.82673034667971, 7.48060911750815, 9.70901634971244, 11.73427734375, 12.9424694847473, 13.3746784965068, 13.2357421875, 12.7119618826274, 11.915492972507, 10.9399543641858, 9.878964964711, 8.82614368112954, 7.87510942048859, 7.11948108983524, 6.65287759621658, 6.56891784667971, 6.86658937181687, 7.16635419640096, 6.99404296875002, 6.05599964800019, 4.78062143656005, 3.77681884765627, 3.53608157376203, 4.08021602433665, 5.31360778808596, 7.11246973837541, 9.24032388720892, 11.43251953125, 13.4014065517295, 14.7673371681472, 15.1276641845703, 14.2582383510391, 12.6489022014869, 10.9679962158203, 9.75085416448453, 9.0007829800791, 8.58808288574221, 8.39744397648294, 8.37111583479402, 8.46573791503908, 8.63304806633875, 8.80517771684158, 8.90935668945314, 8.89664736867907, 8.81344238542531, 8.72996693219792, 8.71644620150294, 8.84310538584642, 9.18016967773439, 9.75419136994435, 10.4170311563395, 10.9768768310547, 11.2818176988904, 11.3395491073107, 11.1976679144458, 10.9037709784256, 10.5054551573802, 10.0503173094396, 9.58595429273386, 9.15996296539293, 8.81994018554689, 8.59736609028363, 8.45925393252248, 8.35650024414064, 8.25015607211298, 8.14189052380496, 8.04352722167971, 7.9622819510895, 7.88693914894327, 7.80167541503908, 7.70636390998746, 7.66366403764866, 7.75193176269533, 8.02304223480711, 8.42294734369139, 8.87111816406252, 9.29001571216296, 9.61406077034774, 9.78066406250002, 9.73304289740776, 9.43764092347821, 8.86670837402346, 8.02806429868355, 7.07180301241053, 6.18358764648439, 5.50698590051584, 5.01718374743824, 4.64727172851564, 4.32370774324191, 3.9464191240302, 3.40870056152346, 2.65237396288315, 1.81337010134474, 1.07614696666217, 0.625162548589407, 0.644874836880392, 1.31974182128908, 2.72426103713761, 4.49308820202078, 6.15091857910158, 7.31349765406045, 7.96077180264762, 8.16373762313075, 7.99339171377755, 7.52073067285568, 6.81675109863283, 5.94934816279549, 4.97401133070543, 3.94312864114323, 2.90908813288949, 1.92427784472478, 1.04108581542971, 0.303318805369995, -0.279541538748006, -0.706594848632789, -0.97964942570594, -1.11134825024139, -1.11704297222617, -1.01208524164729, -0.811826708491789, -0.531619022746674, -0.186813834398973, 0.207237206564291, 0.635182450156097, 1.08167024638942, 1.53134894527724, 1.96886689683253, 2.37887245106828, 2.74601395799746, 3.05493976763304, 3.29029822998801, 3.43673769507534, 3.47890651290801, 3.40145303349899, 3.18902560686128, 2.82627258300783, 2.32474210685319, 1.80358150291815, 1.40883789062502, 1.24192706895459, 1.22573955512141, 1.23853454589846, 1.19894490360604, 1.18709815275374, 1.32349548339846, 1.69501256846599, 2.25402301235768, 2.91927490234377, 3.59590580994331, 4.13461124367068, 4.37247619628908, 4.21366505891221, 3.83065981605578, 3.46302185058596, 3.27137937089376, 3.10062788746946, 2.71672973632814, 1.96454765499548, 1.00454598703956, 0.0760894775390852, -0.640610459444724, -1.20195573392042, -1.72350158691403, -2.29581467500001, -2.90950731694655, -3.53020324707028, -4.08805473649692, -4.37132820358837, -4.1328186035156, -3.22952352600288, -1.93525109898771, -0.628012084960914, 0.369647782565887, 0.957042885997718, 1.08895263671877, 0.760227937008276, 0.12600565272557, -0.618505859374977, -1.31507161103867, -1.95335008800952, -2.55997314453122, -3.1505004166454, -3.69620266758547, -4.15727844238279, -4.50535544688042, -4.75777803016839, -4.94331970214841, -5.08926689631016, -5.21695774049514, -5.34624328613279, -5.44997286902893, -5.31298896249497, -4.67313232421872, -3.38150813148021, -1.74227923992844, -0.172872924804664, 0.97440390026676, 1.60772576812902, 1.70038757324221, 1.27857828978107, 0.580063210779086, -0.104498291015602, -0.564664573677665, -0.910864918573069, -1.3337463378906, -1.95862137039579, -2.64946466116114, -3.20491638183591, -3.48597206026275, -3.60304864905795, -3.72891845703122, -3.98708502469311, -4.30397681935748, -4.55675354003904, -4.61784452951355, -4.34075770560109, -3.57427062988278, -2.24949765007659, -0.626900258447781, 0.950723266601585, 2.1891797207489, 2.98869619598957, 3.29810485839846, 3.10428050319221, 2.54626844215454, 1.80115661621096, 1.01200655279604, 0.185774125380798, -0.704611206054664, -1.63417576909991, -2.3697690416232, -2.62619628906247, -2.24398690286471, -1.56656677851313, -1.06308593749997, -1.0908579291199, -1.55985041387708, -2.2681945800781, -2.96866554587544, -3.23261414880534, -2.58603515624997, -0.762695874972219, 1.67254623074213, 3.94706115722658, 5.42266548547076, 5.99896213508895, 5.71000061035158, 4.65840331333379, 3.22108423732955, 1.84353027343752, 0.852017182237298, 0.0959762022321507, -0.694372558593727, -1.68751312120356, -2.64674602235032, -3.23407592773435, -3.23428993342548, -2.92330485697135, -2.69981994628903, -2.85154705702288, -3.22224847572622, -3.54469909667966, -3.57925393935813, -3.19658852401393, -2.29495849609372, -0.84333610228345, 0.906440005773725, 2.63181457519533, 4.0164248458879, 4.76867802891396, 4.60317382812502, 3.40960670751958, 1.77805017168399, 0.473672485351585, 0.0469481829748752, 0.189576877883182, 0.378564453125022, 0.192462282043727, -0.383996300838115, -1.2645324707031, -2.33851610636087, -3.39791190112925, -4.2103332519531, -4.61272816530235, -4.71938308574716, -4.71391906738279, -4.69834882936245, -4.44825175107145, -3.6575988769531, -2.14778627111777, -0.249910076344181, 1.57750854492189, 2.92953241119335, 3.61556026481781, 3.49857482910158, 2.53221558178429, 1.03274901833895, -0.592901611328102, -1.98914289371234, -3.00570076091654, -3.54363098144528, -3.57275659193186, -3.33796970152446, -3.15292968749997, -3.23871478984764, -3.44607869940615, -3.53319396972653, -3.3417559030466, -3.04755079635133, -2.90988769531247, -3.07104246819147, -3.20515827360849, -2.86934509277341, -1.77003007994546, -0.21090908158143, 1.35500488281252, 2.53457840573967, 3.17419652774106, 3.18012390136721, 2.5236402625063, 1.43608568039738, 0.21381530761721, -0.88490207873928, -1.75414320350437, -2.32607116699216, -2.58918862705569, -2.75735647170382, -3.10077514648435, -3.77623281733465, -4.48686853175048, -4.82240905761716, -4.49557612592178, -3.71107132005855, -2.79659118652341, -2.02285042934524, -1.43263638268451, -1.0117545382346, -0.746010387688899, -0.621209422740763, -0.623157135083581, -0.737659016410729, -0.950520558415586, -1.24754725279152, -1.61454459123193, -2.03731806543018, -2.50167316707965, -2.99341538787371, -3.49835021950576, -4.00228315366915, -4.49101968205727, -4.95036529636351, -5.36612548828122, -5.72687267151477, -6.03224694781237, -6.28465534093321, -6.48650487463647, -6.64020257268133, -6.74815545882698, -6.81277055683262, -6.83645489045742, -6.82161548346057, -6.77065935960126, -6.68599354263866, -6.57002505633198, -6.42516092444039, -6.25380817072308, -6.05837381893923, -5.84126489284804, -5.60488841620868, -5.35165141278035, -5.08396090632222, -4.80422392059349, -4.51484747935334, -4.21823860636095, -3.91680432537552, -3.61295166015622]], "cat": [12], "id": "sd001"}]

# # predictor.accept = 'application/json'
# # predictor.content_type = 'application/json'

# predictor.serializer = json_serializer
# predictor.deserializer = json_deserializer

The predictor runs inference on our input data and returns the predicted digit (as a float value, so we convert to int for display).

In [ ]:
# result = predictor.predict(data)
# print(result)

## Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
# predictor.delete_endpoint()